# River Network

In [ ]:
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pynhd as nhd
from pynhd import NLDI, NHDPlusHR, WaterData
import contextily as ctx

In [ ]:
import sfrmaker

In [ ]:
import flopy

In [ ]:
import basic
m = basic.load_model()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

[PyNHD](https://github.com/cheginit/pynhd) provides access to the Hydro Network-Linked Data Index ([NLDI](https://labs.waterdata.usgs.gov/about-nldi/index.html)) and the [WaterData](https://labs.waterdata.usgs.gov/geoserver/web/wicket/bookmarkable/org.geoserver.web.demo.MapPreviewPage?1) web services for navigating and subsetting [NHDPlus](https://nhdplus.com/NHDPlus) V2 database. Additionally, you can download NHDPlus High Resolution data as well.

First, let's get the watershed geometry of the contributing basin of a USGS station using `NLDI`:

In [ ]:
nldi = NLDI()
station_id = "11467000"

basin = nldi.get_basins(station_id)

The `navigate_byid` class method can be used to navigate NHDPlus in both upstream and downstream of any point in the database. The available feature sources are ``comid``, ``huc12pp``, ``nwissite``, ``wade``, ``wqp``. Let's get ComIDs and flowlines of the tributaries and the main river channel in the upstream of the station.

In [ ]:
flw_main = nldi.navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{station_id}",
    navigation="upstreamMain",
    source="flowlines",
    distance=1000,
)

flw_trib = nldi.navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{station_id}",
    navigation="upstreamTributaries",
    source="flowlines",
    distance=1000,
)

We can get other USGS stations upstream (or downstream) of the station and even set a distance limit (in km):

In [ ]:
st_all = nldi.navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{station_id}",
    navigation="upstreamTributaries",
    source="nwissite",
    distance=1000,
)

st_d20 = nldi.navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{station_id}",
    navigation="upstreamTributaries",
    source="nwissite",
    distance=20,
)

Now, let's get the [HUC12 pour points](https://www.sciencebase.gov/catalog/item/5762b664e4b07657d19a71ea):

In [ ]:
pp = nldi.navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{station_id}",
    navigation="upstreamTributaries",
    source="huc12pp",
    distance=1000,
)

Let's plot the vector data:

In [ ]:
ax = basin.plot(facecolor="none", edgecolor="k", figsize=(8, 8))
st_all.plot(ax=ax, label="USGS stations", marker="*", markersize=300, zorder=4, color="b")
st_d20.plot(
    ax=ax,
    label="USGS stations up to 20 km",
    marker="v",
    markersize=100,
    zorder=5,
    color="darkorange",
)
pp.plot(ax=ax, label="HUC12 pour points", marker="o", markersize=50, color="k", zorder=3)
flw_main.plot(ax=ax, lw=3, color="r", zorder=2, label="Main")
flw_trib.plot(ax=ax, lw=1, zorder=1, label="Tributaries")
ax.legend(loc="best")
ax.set_aspect("auto")
ax.figure.set_dpi(100)
ax.figure.savefig("GIS/nhdplus_navigation.png", bbox_inches="tight", facecolor="w")

Next, we get the slope data for each river segment from NHDPlus VAA database:

In [ ]:
flw_trib.filter(regex = 'id').columns

In [ ]:
vaa = nhd.nhdplus_vaa("GIS/nhdplus_vaa.parquet")

flw_trib["comid"] = pd.to_numeric(flw_trib.nhdplus_comid)
slope = gpd.GeoDataFrame(
    pd.merge(flw_trib, vaa[["comid", "slope"]], left_on="comid", right_on="comid"),
    crs=flw_trib.crs,
)
slope[slope.slope < 0] = np.nan

In [ ]:
slope.plot(
    figsize=(8, 8),
    column="slope",
    cmap="plasma",
    legend=True,
    legend_kwds={"label": "Slope (m/m)"},
)

In [ ]:
# import py3dep

In [ ]:
# x = m.modelgrid.get_xcellcenters_for_layer(1)
# y = m.modelgrid.get_ycellcenters_for_layer(1)



In [ ]:

# elevation = py3dep.elevation_bycoords(list(zip(x.reshape((-1)),y.reshape((-1)))), crs="epsg:2226", source="airmap")

In [ ]:
# np.max(elevation)

In [ ]:
# elev_array = np.reshape(elevation,x.shape)

# plt.imshow(elev_array)

# np.savetxt("GIS/lay_1_top.csv", elev_array*3.28, delimiter=",", fmt ="%.1f")

In [ ]:
import pyproj
import proplot as pplt

In [ ]:
import rioxarray  # noqa: F401
import xarray as xr

Now, let's use [WaterData](https://labs.waterdata.usgs.gov/geoserver/web/) service to get the headwater catchments for this basin:

In [ ]:
lines = sfrmaker.Lines.from_nhdplus_hr(r"C:\GIS\shapefiles\NHD\NHDPLUS_H_1801_HU4_GDB\NHDPLUS_H_1801_HU4_GDB.gdb")

In [ ]:
sfrdata = lines.to_sfr( model=m, model_length_units='feet')

In [ ]:


sfrdata = lines.to_sfr(model = m)



In [ ]:
sfrdata.write_package(filename='nhd_hr_demo.sfr', version='mfnwt')

In [ ]:
sfrdata.write_shapefiles(r'GIS\nhd_hr_demo')

In [ ]:
mod = gpd.read_file('GIS/grid.shp')

In [ ]:
outlets

In [ ]:
cells.query("name=='Russian River'").sort_values('outreach')

In [ ]:
routing = gpd.read_file('GIS/nhd_hr_demo_sfr_routing.shp')
cells = gpd.read_file('GIS/nhd_hr_demo_sfr_cells.shp')
outlets = gpd.read_file('GIS/nhd_hr_demo_sfr_outlets.shp')
model_boundary_5070 = mod.to_crs(epsg=2226)

fig, ax = plt.subplots(figsize=(10,8))
cells.plot('name',ax = ax, zorder = 2, facecolor = 'None')
routing.plot(ax=ax, zorder=3)
outlets.plot(ax=ax, c='red', zorder=4, label='outlets')
model_boundary_5070.plot(ax=ax, facecolor='None', 
                         edgecolor='gray',
                         zorder=1
                        ) 

LegendElement = [
    mpatches.mlines.Line2D([], [], color='red', linewidth=0., marker='o', label='sfr outlet'),
    mpatches.mlines.Line2D([], [], color='#1f77b4', label='sfr routing'),
    mpatches.Patch(facecolor='None', edgecolor='gray', label='Model Boundary\n(active area)')
]

ax.legend(handles=LegendElement, loc='upper left')


f = flopy.plot.PlotMapView(m, ax =  ax)
# f.plot_array(m.bas6.ibound[0])
f.plot_ibound(color_noflow = 'black', alpha = .1)
ctx.add_basemap(ax, crs = 2226)
plt.show()

In [ ]:


flopy_grid = flopy.discretization.StructuredGrid(delr=delr, delc=delc,
                                                 xoff=682688, yoff=5139052,  # lower left corner of model grid
                                                 angrot=0,  # grid is unrotated
                                                 # projected coordinate system of model (UTM NAD27 zone 15 North)
                                                 proj4='epsg:26715'
                                                 )

In [ ]:
flw_mr = nhdp_mr.bybox(basin.geometry[0].bounds)

nhdp_hr = NHDPlusHR("flowline")
flw_hr = nhdp_hr.bygeom(basin.geometry[0].bounds)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8), facecolor="w")

flw_mr.plot(ax=ax1)
ax1.set_title("NHDPlus Medium Resolution")
flw_hr.plot(ax=ax2)
ax2.set_title("NHDPlus High Resolution")
fig.savefig("GIS/hr_mr.png", bbox_inches="tight", facecolor="w")

In [ ]:
flw.columns

Since NHDPlus HR is still at the pre-release stage, let's use the MR flowlines to demonstrate the vector-based accumulation.

Based on a topological sorted river network ``pynhd.vector_accumulation`` computes flow accumulation in the network. It returns a dataframe which is sorted from upstream to downstream that shows the accumulated flow in each node.

PyNHD has a utility called ``prepare_nhdplus`` that identifies such relationship among other things such as fixing some common issues with NHDPlus flowlines. But first we need to get all the NHDPlus attributes for each ComID since `NLDI` only provides the flowlines' geometries and ComIDs which is useful for navigating the vector river network data. For getting the NHDPlus database we use ``WaterData``. The WaterData web service layers are  ``nhdflowline_network``, ``nhdarea``, ``nhdwaterbody``, ``catchmentsp``, ``gagesii``, ``huc08``, ``huc12``, ``huc12agg``, and ``huc12all``. Let's use the ``nhdflowline_network`` layer to get required info.

In [ ]:
comids = [int(c) for c in flw_trib.nhdplus_comid.to_list()]
nhdp_trib = nhdp_mr.byid("comid", comids)
flw = nhd.prepare_nhdplus(nhdp_trib, 0, 0, purge_non_dendritic=False)

To demonstrate the use of routing, let's use `nhdplus_attrs` function to get list of available NHDPlus attributes from [Select Attributes for NHDPlus Version 2.1 Reach Catchments and Modified Network Routed Upstream Watersheds for the Conterminous United States](https://www.sciencebase.gov/catalog/item/5669a79ee4b08895842a1d47) item on `ScienceBase` service. These attributes are in catchment-scale and are available in three categories:

1. Local (`local`): For individual reach catchments,
2. Total (`upstream_acc`): For network-accumulated values using total cumulative drainage area,
3. Divergence (`div_routing`): For network-accumulated values using divergence-routed.

In [ ]:
char_ids = nldi.get_validchars("local")
char_ids.head(5)

Let's get Mean Annual Groundwater Recharge, ``RECHG``, using ``getcharacteristic_byid`` class method and carry out the flow accumulation.

In [ ]:
char = "CAT_RECHG"
area = "areasqkm"

local = nldi.getcharacteristic_byid(comids, "local", char_ids=char)
flw = flw.merge(local[char], left_on="comid", right_index=True)


def runoff_acc(qin, q, a):
    return qin + q * a


flw_r = flw[["comid", "tocomid", char, area]]
runoff = nhd.vector_accumulation(flw_r, runoff_acc, char, [char, area])


def area_acc(ain, a):
    return ain + a


flw_a = flw[["comid", "tocomid", area]]
areasqkm = nhd.vector_accumulation(flw_a, area_acc, area, [area])

runoff /= areasqkm

Since these are catchment-scale characteristic, let's get the catchments then add the accumulated characteristic as a new column and plot the results.

In [ ]:
import networkx as nx
import pylab as plt
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv
import osmnx as ox
from osgeo import ogr, osr
from shapely.geometry import shape
from shapely.geometry import LineString

In [ ]:
G = ox.graph_from_place("Santa Rosa, California, USA", network_type="drive")

In [ ]:
G

In [ ]:
def nodes_to_linestring(path):
    coords_list = [(G.nodes[i]['x'], G.nodes[i]['y']) for i in path ]
    #print(coords_list)
    line = LineString(coords_list)
    
    return(line)

def shortestpath(o_lat, o_long, d_lat, d_long):
    
    nearestnode_origin, dist_o_to_onode = ox.distance.nearest_nodes(G, o_long,  o_lat,  return_dist=True)
    nearestnode_dest, dist_d_to_dnode = ox.distance.nearest_nodes(G,  d_long, d_lat,  return_dist=True)
    
    #Add up distance to nodes from both o and d ends. This is the distance that's not covered by the network
    dist_to_network = dist_o_to_onode + dist_d_to_dnode
    
    shortest_p = nx.shortest_path(G,nearestnode_origin, nearestnode_dest) 
    

    route = nodes_to_linestring(shortest_p) #Method defined above
    
    # Calculating length of the route requires projection into UTM system.  
    inSpatialRef = osr.SpatialReference()
    inSpatialRef.ImportFromEPSG(4326)
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)
    coordTransform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    #route.wkt returns wkt of the shapely object. This step was necessary as transformation can be applied 
    #only on an ogr object. Used EPSG 32643 as Bangalore is in 43N UTM grid zone.
    geom = ogr.CreateGeometryFromWkt(route.wkt)
   
    geom.Transform(coordTransform)
    length = geom.Length()
    
    #Total length to be covered is length along network between the nodes plus the distance from the O,D points to their nearest nodes
    total_length = length + dist_to_network
    #in metres
    
    return(route, total_length )


# coords = [(-103.801086, 40.26772), (-103.80097, 40.270568)]
up = (-122.83164 ,38.44970)
down =  (-122.70897, 38.43966)
coords = [up, down]

out,l = shortestpath(up[1],up[0],down[1],down[0])

In [ ]:
import pynhd
# nx=pynhd.network_tools.nhdflw2nx(flw_hr, id_col='NHDFlowline.comid', toid_col='NHDFlowline.tocomid', edge_attr=None)
G=pynhd.network_tools.nhdflw2nx(flw)

# G.graph['crs'] = flw_mr.crs

x, y = nx.get_node_attributes(G, "x").values(), nx.get_node_attributes(G, "y").values()

nx.set_node_attributes(G, dict(zip(G.nodes(), x)), "x")
nx.set_node_attributes(G, dict(zip(G.nodes(), y)), "y")

nx.shortest_path(

In [ ]:
flw.columns.tolist()

In [ ]:
fls

In [ ]:
flw.set_index('comid').loc[fls[0][:-1],:].plot()

In [ ]:
flw.head()

In [ ]:
flw_hr

In [ ]:
ls.LineString

In [ ]:
import shapely

ls = shapely.geometry.linestring

flw.geometry.apply(lambda x: isinstance(x, ls.LineString)).sort_values()

In [ ]:
nhdp_mr = WaterData("nhdflowline_network")


In [ ]:
vaa = nhd.nhdplus_vaa("input_data/nhdplus_vaa.parquet")

In [ ]:
upf = nldi.navigate_byloc(up,     navigation="upstreamTributaries",
    source="flowlines",)

downf = nldi.navigate_byloc(down,     navigation="upstreamTributaries",
    source="flowlines",)

def gt_(df):
    df["comid"] = pd.to_numeric(df.nhdplus_comid)

    df = gpd.GeoDataFrame(
        pd.merge(df, vaa, left_on="comid", right_on="comid"),
        crs=flw_trib.crs,
    )
    return df

upf = gt_(upf)

In [ ]:
nldi.navigate_byloc(down,     navigation="pp",
    source="flowlines",)

In [ ]:
pynhd.network_resample(upf.set_crs(4326).to_crs(2226), 50)

In [ ]:
import folium

extract basin for downstream  
find nearest node to upstream
- get node
- use downstream node and route to upstream node
- get nodes in route

In [ ]:
sort,route, G = pynhd.network_tools.topoogical_sort(flw.rename(columns = {'comid':'ID','tocomid':'toID'}))

In [ ]:
flw.geometry.centroid

In [ ]:
flw.geometry.centroid.x

In [ ]:
pos = flw.assign(x =flw.geometry.centroid.x, y =flw.geometry.centroid.y).set_index('comid').loc[:,['x','y']].T.to_dict()
pos = {k:(v['x'], v['y']) for k,v in pos.items()}

In [ ]:
fff = pynhd.nhdflw2nx(flw)

flw.sort_values('tocomid')

In [ ]:
import contextily as ctx
import cartopy
import cartopy.crs as ccrs

In [ ]:
tb = flw.to_crs(epsg=4326).total_bounds
box = [tb[0], tb[2], tb[1], tb[3]]

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_extent(box, crs=ccrs.PlateCarree())

# Put a background image on for nice sea rendering.
# ax.stock_img()



nx.draw_networkx(pynhd.nhdflw2nx(flw.drop(62)), pos = pos,ax= ax)
ctx.add_basemap(ax, crs = 4326)

In [ ]:
G.add_node(

In [ ]:
def near(pt, flow):
    print(pt)
    g = gpd.points_from_xy([pt[0]], [pt[1]])
    gdf = gpd.GeoDataFrame(["meas"], columns = ['h'],geometry = g, crs = 4326).to_crs(2226)
    
    nearest = gpd.sjoin_nearest(gdf, flow.to_crs(2226))
    
    n = nearest.loc[:,'comid'].values[0]
    print(n)
        
    return n, gdf

def route(G, up, down):

    upcom, up_df = near(up, flw)
    dcom, down_df = near(down, flw)    
    
    route_df = nx.shortest_path(G,dcom, upcom)
    
    route_df = flw.set_index('comid').loc[route_df]
    return route_df, up_df, down_df



route_df, up_df, down_df = route(G, up, down)
m = plot_route(route_df.reset_index(), up_df, down_df )
m

In [ ]:
def plot_route(route_df, up, down):
    m  = route_df.loc[:,['comid','gnis_name','lengthkm','reachcode','flowdir','ftype','geometry']].explore()
    

    # up.explore(m = ax,popup = ['Top'])
    # up.explore(m=m)
    # down.explore(m=m)
    folium.Marker((up.to_crs(4326).geometry.y,up.to_crs(4326).geometry.x) ).add_to(m)
    folium.Marker((down.to_crs(4326).geometry.y,down.to_crs(4326).geometry.x) ).add_to(m)
    # folium.Marker(up[::-1], ).add_to(ax)
    # ax
    # ax.scatter(down0], down[1])
    
    return m



In [ ]:
pynhd.network_tools.network_resample(upf.set_crs(4326).to_crs(2226),100.)

In [ ]:
dire = r'c:\GSP\SRP\Seepage_Runs'
f = os.path.join(dire, 'flow_routing.csv')
nw = pd.read_csv(f)
locs = gpd.read_file(r"C:\GSP\srp\Seepage_runs\GIS\all_obs_locs.shp").to_crs(4326)
nw

In [ ]:
G = nx.Graph()

for _,row in nw.iterrows():
    row = row.dropna()
    G.add_node(row['node'])
    
    for i in range(row.shape[0]-2):
        i = i+1
        print(f'from{i}')
        G.add_edge(row['node'], row[f'from{i}'])


In [ ]:
tb = locs.to_crs(epsg=4326).total_bounds
box = [tb[0], tb[2], tb[1], tb[3]]

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_extent(box, crs=ccrs.PlateCarree())

pos = locs.assign(x =locs.geometry.centroid.x, y =locs.geometry.centroid.y).set_index('name').loc[:,['x','y']].T.to_dict()
pos = {k:(v['x'], v['y']) for k,v in pos.items()}
nx.draw_networkx(G, pos = pos)

ctx.add_basemap(ax, crs = 4326)

In [ ]:
upcom

In [ ]:
dcom

In [ ]:
res = pynhd.network_tools.network_resample(flw.to_crs(2226),50)

In [ ]:
pynhd.topoogical_sort(flw)

In [ ]:
catchments = wd_cat.byid("featureid", comids)

c_local = catchments.merge(local, left_on="featureid", right_index=True)
c_acc = catchments.merge(runoff, left_on="featureid", right_index=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8), facecolor="w")

cmap = "viridis"
norm = plt.Normalize(vmin=c_local.CAT_RECHG.min(), vmax=c_acc.acc_CAT_RECHG.max())

c_local.plot(ax=ax1, column=char, cmap=cmap, norm=norm)
flw.plot(ax=ax1, column="streamorde", cmap="Blues")
ax1.set_title("Groundwater Recharge (mm/yr)")

c_acc.plot(ax=ax2, column=f"acc_{char}", cmap=cmap, norm=norm)
flw.plot(ax=ax2, column="streamorde", cmap="Blues")
ax2.set_title("Accumulated Groundwater Recharge (mm/yr)")

cax = fig.add_axes(
    [
        ax2.get_position().x1 + 0.01,
        ax2.get_position().y0,
        0.02,
        ax2.get_position().height,
    ]
)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
fig.colorbar(sm, cax=cax)
fig.savefig("_static/flow_accumulation.png", bbox_inches="tight", facecolor="w")